In [3]:
import math
import numpy as np
import cv2
import time
import mediapipe as mp
import pandas as pd

SCALE_FACTOR = 1

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

def main():
    global image
    #cap = cv2.VideoCapture("/Users/calvin/Downloads/tv_human_interactions_videos/handShake_0029.avi")
    #cap = cv2.VideoCapture("/Users/calvin/Downloads/tv_human_interactions_videos/negative_0092.avi")
    cap = cv2.VideoCapture("/Users/calvin/Downloads/tv_human_interactions_videos/kiss_0009.avi")
    with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
        pose.enable_segmentation = False
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring emtpty camera frame.")
                break

            image = resize_image(image)
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

            keyPoints = storeKeyPoints(results, image.shape)
            image = get_centroid(image, keyPoints)
            stencil = drawStencil(keyPoints, image.shape)
            stencil = drawLines(stencil, keyPoints, list(mp_pose.POSE_CONNECTIONS))
            stack = stack_image([image, stencil])
  
            cv2.imshow("stack", cv2.flip(stack, 1))
            if cv2.waitKey(100) & (0xFF == 27 | 0xFF == ord("q")):
                cv2.destroyAllWindows()
                break
        cap.release()
        

In [4]:
_empty = {"x" : 0, "y" : 0}


def storeKeyPoints(results, shape):
    keyPoints = {}
    for idx, lm in enumerate(results.pose_landmarks.landmark):
        keyPoints[idx] = { "x" : int(lm.x * shape[1]),
                           "y" : int(lm.y * shape[0]),
                           "z" : lm.z,
                           "v" : lm.visibility}
    keyPoints[33] = {"x" : (keyPoints.get(24).get("x", _empty) + keyPoints.get(23).get("x")) // 2,
                     "y" : (keyPoints.get(24).get("y", _empty) + keyPoints.get(23).get("y")) // 2}
    return keyPoints


def drawStencil(keyPoints, imageDimension):
    blankImage = np.zeros(imageDimension)
    image = drawCircles(blankImage, keyPoints, range(1, 33))
    return image
    

def drawCircles(image, keyPoints, circle_points=[], small=False):
    for points in circle_points:
        if isinstance(keyPoints, dict):
            _temp = (keyPoints[points]["x"], keyPoints[points]["y"])
        elif isinstance(keyPoints, tuple):
            _temp = keyPoints
        if small:
            image = cv2.circle(image, _temp, radius=1, color=(255, 255, 255), thickness=2)     
        else:
            image = cv2.circle(image, _temp, radius=5, color=(0, 0, 255), thickness=-10)
            image = cv2.circle(image, _temp, radius=7, color=(255, 255, 255), thickness=2)   
    return image
    
def drawLines(image, keyPoints, line_points=[]):
    for line in line_points:
        line_start = keyPoints.get(line[0], _empty)
        line_start = (line_start["x"], line_start["y"])
        line_end = keyPoints.get(line[1], _empty)
        line_end = (line_end["x"], line_end["y"])
        image = cv2.line(image, line_start, line_end, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
    return image

def find_angleDistance(keyPoints, image, points, annotate=True):
    ret = []
    for point in points:
        _temp = {"angle" : None, "distance" : None, "midpoint" : None}
        p1, p2 = point
        p1 = keyPoints.get(p1, None)
        p2 = keyPoints.get(p2, None)
        if p1 != None and p2 != None:
            y2y1 = p2.get("y") - p1.get("y")
            x2x1 = p2.get("x") - p1.get("x")
            if x2x1 > 0:
                _temp["angle"] = math.degrees(math.atan(y2y1/x2x1))
                _temp["distance"] = int(math.sqrt((y2y1**2) + (x2x1**2)))
                _temp["midpoint"] = ((p2.get("x") + p1.get("x"))//2, (p2.get("y") + p1.get("y"))//2)
                drawCircles(image, _temp["midpoint"], [0], small=True)
                ret.append(_temp)
    return ret
    
def annotateMeasurements(image, measurements):
    for measure in measurements:
        if measure.get("midpoint", None):
            x, y = measure.get("midpoint")
            image = cv2.putText(image, str(measure.get("distance", "null")), (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                                fontScale=1, thickness=2, color=(255, 255, 255), lineType=cv2.LINE_AA)
    return image

def get_centroid(image, keyPoints):
    measurements = find_angleDistance(keyPoints, image, [(33, 16)])
    image = annotateMeasurements(image, measurements)
    image = drawLines(image, keyPoints, [(33, 0), (33, 16), (33, 15)])
    image = drawCircles(image, keyPoints, [0, 33, 16, 15])
    return image

def stack_image(images=[]):
    out = images[0].astype(np.uint8)
    for image in images[1:]:
        out = cv2.vconcat((out, image.astype(np.uint8)))
    return out
        
    
def resize_image(image, fixed=False):
    return cv2.resize(image, (int(image.shape[1] * SCALE_FACTOR), int(image.shape[0] * SCALE_FACTOR)), interpolation=cv2.INTER_AREA)
    
main()

Ignoring emtpty camera frame.
